In [1]:
!pip install PyGithub dspy openai

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import os
import openai
from github import Github
import dspy

In [3]:
github_token = os.getenv('GITHUB_TOKEN')  
openai.api_key = os.getenv('OPENAI_API_KEY')

g = Github(github_token)

In [4]:
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop") 
base_directory = os.path.join(desktop_path, "repositories/") 

if not os.path.exists(base_directory):
    os.makedirs(base_directory)
    print(f"Created directory: {base_directory}") 
else:
    print(f"Directory already exists: {base_directory}")  

Directory already exists: C:\Users\jimmy\Desktop\repositories/


In [5]:
def search_repositories(keyword, language="python", stars=">=100", max_repos=10):
    """Search for GitHub repositories based on a keyword."""
    query = f"{keyword} language:{language} stars:{stars}"
    results = []
    try:
        for repo in g.search_repositories(query=query, sort="stars", order="desc")[:max_repos]:
            results.append({
                'full_name': repo.full_name,
                'html_url': repo.html_url,
                'description': repo.description
            })
        print(f"Found {len(results)} repositories for keyword '{keyword}'.")
    except Exception as e:
        print(f"GitHub search error: {e}")
    return results

In [6]:
def fetch_repository_details(repo_name):
    """Fetch the README content of a specified GitHub repository."""
    try:
        repo = g.get_repo(repo_name)
        return repo.get_readme().decoded_content.decode("utf-8")
    except Exception as e:
        print(f"Error fetching {repo_name}: {e}")
        return None

In [7]:
def analyze_readme_keywords(readme_content, keywords):
    """Analyze the README content for specified keywords."""
    data = dspy.DataFrame({'content': [readme_content]})
    return {keyword: data['content'].str.contains(keyword).sum() for keyword in keywords}

In [8]:
def process_repository(repo_full_name, keywords):
    """Process a repository: fetch README and analyze keywords."""
    readme_content = fetch_repository_details(repo_full_name)
    if readme_content:
        keyword_counts = analyze_readme_keywords(readme_content, keywords)
        print(f"Keyword counts for {repo_full_name}: {keyword_counts}")

In [9]:
if __name__ == "__main__":
    keyword = input("Enter a keyword to search for GitHub repositories: ")
    keywords = input("Enter keywords to analyze in the README (comma-separated): ").split(',')
    keywords = [k.strip() for k in keywords]
    
    repos = search_repositories(keyword)
    for repo in repos:
        process_repository(repo['full_name'], keywords)


Enter a keyword to search for GitHub repositories:  stable diffusion
Enter keywords to analyze in the README (comma-separated):  help


Found 10 repositories for keyword 'stable diffusion'.


AttributeError: module 'dspy' has no attribute 'DataFrame'

In [ ]:
if __name__ == "__main__":
    keyword = input("Enter a keyword to search for GitHub repositories: ")
    keywords = input("Enter keywords to analyze in the README (comma-separated): ").split(',')
    keywords = [k.strip() for k in keywords]
    
    repos = search_repositories(keyword)
    for repo in repos:
        process_repository(repo['full_name'], keywords)
